#**Đề tài: Phân tích, thống kê video thịnh hành trên youtube dựa trên hơn 40.000 video**


# 1. Gather data

## 1.1 Problem Statement
**Trần Phan Minh Tiến**

    Question 1: Có phải số lượt xem lớn là điều kiện tiên quyết để một video trở thành thịnh hành không?

    Question 2: Biểu đồ cột khối lượng giao dịch theo năm 2015 - 2025
**Trần Minh Thuận**

    Question 3: Biểu đồ thể hiện khối lượng giao dịch theo từng tháng năm 2024

    Question 4: Vẽ biểu đồ lợi nhuận hàng tháng năm 2024
**Hoàng Văn Đức**

    Question 5: Vẽ biểu đồ thể hiện Số lượng chứng khoán bán theo quý (2015, 2020, 2024)

    Question 6: Vẽ biểu đồ thể hiện Giá cao trung bình theo ngày trong tuần năm 2024

## 1.2 Import libraries:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import pandas as pd

drive_path = '/content/drive/MyDrive/Colab Notebooks/data-ytb'

files = os.listdir(drive_path)

csv_files = [file for file in files if file.endswith('.csv')]

for file in csv_files:
    file_path = os.path.join(drive_path, file)
    print(f"Đang đọc tệp: {file_path}")
    try:
        df = pd.read_csv(file_path, encoding='ISO-8859-1')
        print(df.head())
    except UnicodeDecodeError as e:
        print(f"Lỗi khi đọc tệp {file}: {e}")


Đang đọc tệp: /content/drive/MyDrive/Colab Notebooks/data-ytb/CAvideos.csv
      video_id trending_date  \
0  n1WpP7iowLc      17.14.11   
1  0dBIkQ4Mz1M      17.14.11   
2  5qpjK5DgCt4      17.14.11   
3  d380meD0W0M      17.14.11   
4  2Vv-BfVoq4g      17.14.11   

                                               title channel_title  \
0        Eminem - Walk On Water (Audio) ft. BeyoncÃ©    EminemVEVO   
1                      PLUSH - Bad Unboxing Fan Mail     iDubbbzTV   
2  Racist Superman | Rudy Mancuso, King Bach & Le...  Rudy Mancuso   
3                           I Dare You: GOING BALD!?      nigahiga   
4        Ed Sheeran - Perfect (Official Music Video)    Ed Sheeran   

   category_id              publish_time  \
0           10  2017-11-10T17:00:03.000Z   
1           23  2017-11-13T17:00:00.000Z   
2           23  2017-11-12T19:05:24.000Z   
3           24  2017-11-12T18:01:41.000Z   
4           10  2017-11-09T11:04:14.000Z   

                                              

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 1.3 Xem thông tin của Dataset

# 2. Assess data:

---



## 2.1 Thêm cột

##**Nhận xét dữ liệu:**
-
-
-

# 3. Clean data:

#**4.Save new after clean**

# **5.Answer the research question (Visuazation)**

##**Trần Phan Minh Tiến**
### **Question 1**:Biểu đồ đường giá đóng cửa trong 10 năm qua theo thời gian

##**Trần Phan Minh Tiến**
### **Question 2**:Biểu đồ cột khối lượng giao dịch theo năm 2015 - 2025

##**Trần Minh Thuận**
### **Question 3**:Biểu đồ thể hiện khối lượng giao dịch theo từng tháng năm 2024

##**Trần Minh Thuận**
### **Question 4**:Vẽ biểu đồ lợi nhuận hàng tháng năm 2024

##**Hoàng Văn Đức**
### **Question 5**:Vẽ biểu đồ thể hiện Số lượng chứng khoán bán theo quý (2015, 2020, 2024)

##**Hoàng Văn Đức**
### **Question 6**:Vẽ biểu đồ thể hiện Giá cao trung bình theo ngày trong tuần năm 2024

###**Nhận xét:**
- Qua biểu đồ ta có thể thấy ...

- AAPL biến động ... giảm ... tăng ... giảm... tăng ...

- META biến động ... giảm ... tăng ... giảm... tăng ...

###**Lý do (Khuyến nghị)**
- Lý do cho sự biến đổi là do vào năm ... có vấn đề ... năm ... có ....
